In [1]:
import numpy as np
import pandas as pd
import math
import sys
from ipynb.fs.full.Formulas import *
import random

Bhaichara On Top


# Calculating number of slots for a flow

In [2]:
# Datarate might be wrong, review it later
def calculate_number_of_slots():
    calculate_datarate_without_interference()
    
    for flow_index in range(1, number_of_flows + 1):
        sender = flows[flow_index][2]
        receiver = flows[flow_index][3]
        number_of_slots_for_each_flow[flow_index] = ((throughputs[flow_index] * number_of_time_slots) // datarate_without_interference[flow_index])

## Vehicle and UAV class declared here

In [3]:
class Vehicle:
    def __init__(self, x, y, velocity):
        self.x = x
        self.y = y
        self.velocity = velocity
        self.type = 'vehicle'
        
    def calculate_position(self, time):
        current_x = self.x + (self.velocity * time)
        return current_x, self.y
    
class UAV:
    def __init__(self, x, y, theta, height = height_of_uav, velocity = speed_of_uav, radius = radius_of_uav):
        self.x = x
        self.y = y
        self.height = height
        self.radius = radius
        self.velocity = velocity
        self.theta = theta
        self.type = 'uav'

    def calculate_position(self, time):
        angle = (self.theta + ((self.velocity * time) / self.radius)) % (2 * math.pi)
        current_x = self.x + (self.radius * math.cos(angle))
        current_y = self.y + (self.radius * math.sin(angle))
        return current_x, current_y 


## Relay class models the relaying environment. Environment consists of vehicles and uavs in different positions

In [4]:
class Relay:    
    def isbetween(self, middle, x, y):  # Check whether x < middle < y
        left = min(x, y)
        right = max(x, y)
        return middle > left and middle < right
        
    def no_of_obstacles(self, vehicle1, vehicle2, time): # Count no of vehicles between vehicle 1 and vehicle 2 lying in same lane
        vehicle1_x , vehicle1_y = vehicle1.calculate_position(time)
        vehicle2_x, vehicle2_y = vehicle2.calculate_position(time)
        ans = 0
        
        for vehicle in list_of_vehicles:
            current_x, current_y = vehicle.calculate_position(time)
            
            if current_y == vehicle1_y and self.isbetween(current_x, vehicle1_x, vehicle2_x):
                ans += 1
        return ans    

    # Vehicle 1 = sender and Vehicle 2 = receiver
    def determine_relay_type(self, flow_index, time): # Determining type of relaying needed to take place between a flow of sender and receiver and returning candidate relay set if relay is required 
        vehicle1 = flows[flow_index][0]
        vehicle2 = flows[flow_index][1]
        vehicle1_x, vehicle1_y = vehicle1.calculate_position(time)
        vehicle2_x, vehicle2_y = vehicle2.calculate_position(time)
        
        if vehicle1_y == vehicle2_y:    #Sender and Receiver on same lane
            obstacles = self.no_of_obstacles(vehicle1, vehicle2, time)
            if obstacles == 0: # If no obstacle, no need to relay
                return 'dont_relay', set()
            elif obstacles == 1: # Only one obstacle, use it to relay
                return 'same_lane_relay', self.candidate_relay_set_single_same_lane(flow_index, time) 
            else: # Multiple obstacles, pick relaying vehicle 
                return 'adjacent_lane_relay', self.candidate_relay_set_multiple_same_lane(flow_index, time)
        elif abs(vehicle1_y - vehicle2_y) == 1: # Sender and Receiver on adjacent lanes
            return 'dont_relay', set()
        else:     # Sender and Receiver are multiple lanes(>1) apart
            return 'middle_lane_relay', self.candidate_relay_set_middle_lane(flow_index, time)
    
    def candidate_relay_set_middle_lane(self, flow_index, time):
        vehicle1 = flows[flow_index][0]
        vehicle2 = flows[flow_index][1]
        candidate_relays = set()
        vehicle1_x_start, vehicle1_y_start = vehicle1.calculate_position(time)
        vehicle2_x_start, vehicle2_y_start = vehicle2.calculate_position(time)

        if vehicle1_y_start > vehicle2_y_start:
            return self.candidate_relay_set_middle_lane(flow_index, time)
        
        vehicle1_x_end, vehicle1_y_end = vehicle1.calculate_position(time + number_of_slots_for_each_flow[flow_index] * slot_duration)
        vehicle2_x_end, vehicle2_y_end = vehicle2.calculate_position(time + number_of_slots_for_each_flow[flow_index] * slot_duration)
        
        for vehicle in list_of_vehicles:
            current_x_start, current_y_start = vehicle.calculate_position(time)
            current_x_end, current_y_end = vehicle.calculate_position(time + number_of_slots_for_each_flow[flow_index] * slot_duration)
            
            if not self.isbetween(current_y_start, vehicle1_y_start, vehicle2_y_start): 
                continue

            m = current_y_start - vehicle1_y_start
            n = vehicle2_y_start - current_y_start
            D1x, D1y = section_formula(vehicle1_x_start, vehicle1_y_start, vehicle2_x_start, vehicle2_y_start, m, n)
            D2x, D2y = section_formula(vehicle1_x_end, vehicle1_y_end, vehicle2_x_end, vehicle2_y_end, m, n)
            
            if current_x_end <= D1x or D2x <= current_x_start: # No overlap=No add
                continue
                
            start_boundary = max(current_x_start, D1x)
            end_boundary = min(current_x_end, D2x)
            epsilon = (end_boundary - start_boundary) / (current_x_end - current_x_start)
            
            if epsilon >= 0.5:
                candidate_relays.add(vehicle)
                
        # If candidate_relays.length()==0 till now, it means there's no obstacle in middle lanes and no need to relay. so dont add uav and return empty set
        if len(candidate_relays):
            candidate_relays = candidate_relays.union(self.get_uav(flow_index, time))
        return candidate_relays

    def candidate_relay_set_single_same_lane(self, flow_index, time):
        vehicle1 = flows[flow_index][0]
        vehicle2 = flows[flow_index][1]
        candidate_relays = set()
        vehicle1_x, vehicle1_y = vehicle1.calculate_position(time)
        vehicle2_x, vehicle2_y = vehicle2.calculate_position(time)
        
        for vehicle in list_of_vehicles:
            current_x, current_y = vehicle.calculate_position(time)
            
            if self.isbetween(current_x, vehicle1_x, vehicle2_x):
                candidate_relays.add(vehicle)
                break
                
        candidate_relays = candidate_relays.union(self.get_uav(flow_index, time))
        return candidate_relays

    def candidate_relay_set_multiple_same_lane(self, flow_index, time):
        vehicle1 = flows[flow_index][0]
        vehicle2 = flows[flow_index][1]
        candidate_relays = set()
        vehicle1_x_start, vehicle1_y_start = vehicle1.calculate_position(time)
        vehicle2_x_end, vehicle2_y_end = vehicle2.calculate_position(time + number_of_slots_for_each_flow[flow_index] * slot_duration)
        D3x = vehicle1_x_start
        D4x = vehicle2_x_end
        
        for vehicle in list_of_vehicles:
            current_x_start, current_y_start = vehicle.calculate_position(time)
            current_x_end, current_y_end = vehicle.calculate_position(time + number_of_slots_for_each_flow[flow_index] * slot_duration)
            
            if current_y_start == (vehicle1_y_start - 1) or current_y_start == (vehicle1_y_start + 1):
                if current_x_end <= D3x or D4x <= current_x_start:
                    continue
                
                start_boundary = max(current_x_start, D3x)
                end_boundary = min(current_x_end, D4x)
                epsilon = (end_boundary - start_boundary) / (current_x_end - current_x_start)
                
                if epsilon >= 0.5:
                    candidate_relays.add(vehicle)
                    
        candidate_relays = candidate_relays.union(self.get_uav(flow_index, time))
        return candidate_relays

    def get_uav(self, flow_index, time): # Function to return list of possible UAVs available in flow range for relaying
        vehicle1 = flows[flow_index][0]
        vehicle2 = flows[flow_index][1]
        candidate_uavs = set()
        vehicle1_x, vehicle1_y = vehicle1.calculate_position(time)
        vehicle2_x, vehicle2_y = vehicle2.calculate_position(time)
        for i in range(1, number_of_uavs + 1):
            current_x, current_y = list_of_uavs[i].calculate_position(time)
            distance_s = euclidean_distance(current_x, current_y, vehicle1_x, vehicle1_y)
            distance_r = euclidean_distance(current_x, current_y, vehicle2_x, vehicle2_y) 
            
            if distance_s <= coverage_radius and distance_r <= coverage_radius:
                candidate_uavs.add(list_of_uavs[i])
                
        return candidate_uavs

In [5]:
# Declare list of vehicles and UAVs objects
list_of_vehicles = [
        Vehicle(10, 1, 75),    
        Vehicle(20, 1, 70),    
        Vehicle(30, 1, 65),     
        Vehicle(40, 1, 60),    
        Vehicle(50, 1, 55),    
        Vehicle(60, 1, 70),    
        Vehicle(70, 1, 72),    
        Vehicle(80, 1, 24),    
        Vehicle(90, 1, 25),   
        Vehicle(100, 1, 50),    
        Vehicle(110, 1, 45),    
        Vehicle(120, 1, 40),    
        Vehicle(130, 1, 35),    
        Vehicle(140, 1, 30),
        Vehicle(10, 3, 75),    
        Vehicle(20, 3, 70),    
        Vehicle(30, 3, 65),    
        Vehicle(40, 3, 60),    
        Vehicle(50, 3, 55),    
        Vehicle(60, 3, 70),    
        Vehicle(70, 3, 72),    
        Vehicle(80, 3, 24),    
        Vehicle(90, 3, 25),   
        Vehicle(100, 3, 50),    
        Vehicle(110, 3, 45),    
        Vehicle(120, 3, 40),    
        Vehicle(130, 3, 35),    
        Vehicle(140, 3, 30),
        Vehicle(10, 2, 75),    
        Vehicle(20, 2, 70),    
        Vehicle(30, 2, 65),    
        Vehicle(40, 2, 60),    
        Vehicle(50, 2, 55),    
        Vehicle(60, 2, 70),    
        Vehicle(70, 2, 72),    
        Vehicle(80, 2, 24),    
        Vehicle(90, 2, 25),   
        Vehicle(100, 2, 50),    
        Vehicle(110, 2, 45),    
        Vehicle(120, 2, 40),    
        Vehicle(130, 2, 35),    
        Vehicle(140, 2, 30),
        Vehicle(15, 1, 75),    
        Vehicle(25, 1, 70),    
        Vehicle(35, 1, 65),    
        Vehicle(45, 1, 60),    
        Vehicle(55, 1, 55),    
        Vehicle(65, 1, 70),    
        Vehicle(75, 1, 72),    
        Vehicle(85, 1, 24),    
        Vehicle(95, 1, 25),   
        Vehicle(105, 1, 50),    
        Vehicle(115, 1, 45),    
        Vehicle(125, 1, 40),    
        Vehicle(135, 1, 35),    
        Vehicle(145, 1, 30),
        Vehicle(15, 2, 75),    
        Vehicle(25, 2, 70),    
        Vehicle(35, 2, 65),    
        Vehicle(45, 2, 60),    
        Vehicle(55, 2, 55),    
        Vehicle(65, 2, 70),    
        Vehicle(75, 2, 72),    
        Vehicle(85, 2, 24),    
        Vehicle(95, 2, 25),   
        Vehicle(105, 2, 50),    
        Vehicle(115, 2, 45),    
        Vehicle(125, 2, 40),    
        Vehicle(135, 2, 35),    
        Vehicle(145, 2, 30),
        Vehicle(15, 3, 75),    
        Vehicle(25, 3, 70),    
        Vehicle(35, 3, 65),    
        Vehicle(45, 3, 60),    
        Vehicle(55, 3, 55),    
        Vehicle(65, 3, 70),    
        Vehicle(75, 3, 72),    
        Vehicle(85, 3, 24),    
        Vehicle(95, 3, 25),   
        Vehicle(105, 3, 50),    
        Vehicle(115, 3, 45),    
        Vehicle(125, 3, 40),    
        Vehicle(135, 3, 35),    
        Vehicle(145, 3, 30)
]

list_of_uavs = [
        UAV(0, 0, math.pi),  # Dummy Object (no use)
        UAV(10, 1, math.pi / 4),
        UAV(500, 2, math.pi / 3),
        UAV(200, 3, math.pi / 6)
]

# Create a list to store flows
flows = [[list_of_vehicles[0],list_of_vehicles[1],0,1]]

# Loop through the list and form flow pairs (sender and receiver pairs)
for i in range(1,len(list_of_vehicles)):
    if len(flows) == (number_of_flows + 1):
        break

    flows.append([list_of_vehicles[0], list_of_vehicles[i], 0, i])

# Convert the list of pairs to a NumPy array
flows = np.array(flows)

for i in range(1, len(flows)):
    for j in range(1, len(flows)):
        flow_sender = flows[i][0]
        flow_receiver = flows[j][1]

        sender_x,sender_y = flow_sender.calculate_position(0)
        receiver_x,receiver_y = flow_receiver.calculate_position(0)
        distance_matrix[i,j] = euclidean_distance(sender_x, sender_y, receiver_x, receiver_y)

# u2v and v2u distance calculation

for i in range(1, number_of_uavs + 1):
    for j in range(1, number_of_flows + 1):
        flow_sender = flows[j][0]
        flow_receiver = flows[j][1]
        sender_x, sender_y = flow_sender.calculate_position(0)
        receiver_x, receiver_y = flow_receiver.calculate_position(0)
        current_uav = list_of_uavs[i]
        uav_x,uav_y = current_uav.calculate_position(0)
        distance_u2v[i,j] = euclidean_distance(uav_x, uav_y, receiver_x, receiver_y)
        distance_v2u[j,i] = euclidean_distance(sender_x, sender_y, uav_x, uav_y)

# Testing segment to store standard output in a file (dont run unless required)

In [6]:
# # Open a file in write mode
# with open('output.txt', 'w') as f:
#     # Redirect stdout to the file
#     sys.stdout = f
    
#     # Your print statements
#     temp= Relay()
#     for i in range(5):
#         time = i*slot_duration
#         for j in flows:
#             print("Sender:",j[0].calculate_position(time),"Receiver:",j[1].calculate_position(time))
#             relay_type,candidate_set=temp.determine_relay_type(j[0],j[1],time)
#             for k in candidate_set:
#                 print(k.calculate_position(time))
#             # print(temp.determine_relay_type(j[0],j[1],i*slot_duration))
    
#     # Remember to restore stdout to its default value after writing
#     sys.stdout = sys.__stdout__

# # File is automatically closed after the `with` block

# Creating flows and storing their candidate sets

### This function picks up random candidate out of a particular relay set

In [7]:
def random_relay_candidate(my_set):
    if not my_set:
        return -1
    else:
        return random.choice(list(my_set))

# Calling Functions

In [8]:
calculate_channel_power_gain_distribution()
calculate_received_signal_power()
calculate_sinr_and_datarate_u2v()
calculate_sinr_and_datarate_v2u()
calculate_sinr_and_datarate_v2v()
calculate_number_of_slots()
flows_candidate_relay_set_pairs={}
RelayObj = Relay()

### Mapping flows to their respective (relay type,candidate set)

In [9]:

# for current_time_slot in range(number_of_time_slots):
#     time = current_time_slot * slot_duration
#     for flow in flows:
#         flow_with_time = [flow[0].calculate_position(time), flow[1].calculate_position(time), time]
#         flow_with_time = str(flow_with_time)
#         relay_type,candidate_set = RelayObj.determine_relay_type(flow[0],flow[1],time)
#         flows_candidate_relay_set_pairs[flow_with_time] = [relay_type,candidate_set,random_relay_candidate(candidate_set)]
for current_time_slot in range(number_of_time_slots):
    time = current_time_slot * slot_duration
    for flow_index in range(1, len(flows)):
        if (current_time_slot + number_of_slots_for_each_flow[flow_index]) > number_of_time_slots:
            break
            
        flow_with_time = [flow_index, time]
        flow_with_time = str(flow_with_time)
        relay_type,candidate_set = RelayObj.determine_relay_type(flow_index, time)
        flows_candidate_relay_set_pairs[flow_with_time] = [relay_type, candidate_set, random_relay_candidate(candidate_set)]